In [1]:
import sys
sys.path.append("../../DeepFakeVideoDetection/src/")

from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
import numpy as np
# from resnet_convlstm import ResNet
from Utility_functions import FreezeBatchNormalization
from datetime import datetime as dt
from DFVDSequence import DFVDSequence
import matplotlib.pyplot as plt
print("yolo")
from sklearn.metrics import classification_report
# import pandas as pd
# import shutil
from tensorflow.python.keras import layers
# from src.xception_convlstm import Xception
# print(cv2.__version__)
# import random
# random.seed(32)

dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

loaded
yolo




2023-08-21 14:23:52.305619: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-08-21 14:23:52.463016: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2023-08-21 14:23:52.468389: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5876c10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-08-21 14:23:52.468680: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-08-21 14:23:52.512483: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


2023-08-21 14:23:52.832086: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x586eaf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-21 14:23:52.832179: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-08-21 14:23:52.833795: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:41:00.0
2023-08-21 14:23:52.834510: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/binh/anaconda3/envs/deepfacelab1/lib/python3.7/site-packages/cv2/../../lib64:
2023-08-21 14:23:52.835047: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load d

In [2]:
tf.__version__

'1.15.0'

In [18]:
ALL_DATASET = ["DeepFaceLab", "Dfaker", "Faceswap", "FOM_Animation", "FOM_Faceswap", "FSGAN", "LightWeight", "fake"]
import natsort
import random
from tqdm import tqdm
import glob
import os
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras import layers
# from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
random.seed(32)
# import pandas as pd
from tensorflow.python.keras.callbacks import Callback

def create_sequence(dirs, fpvpb, no_frames):
    random.seed(35)
    count_real = 0
    count_fake = 0
    folders=[]
    for directory in dirs:
        print("Including ", directory)
        folders += [i for i in sorted(glob.glob(os.path.join(directory, '*'))) if os.path.isdir(i)]
    random.shuffle(folders)
    total_folders = len(folders)
    X = {}
    y = []
    print('Total Video Folders Found (Real + Fake):', total_folders)
    pre_folder = -1
    i = 0
    _downsampled = []

    for (idx, folder) in tqdm(enumerate(folders)):
        dir_name = os.path.dirname(folder)
        folder_name = os.path.basename(dir_name)
        # print("video: ", folder)
        # _downsampled = [x for x in sorted(glob.glob(os.path.join(folder, "*.png")))][:no_frames]
        _downsampled = [x for x in sorted([os.path.join(folder, u) for u in os.listdir(folder) if u .endswith(".png")])][:no_frames]

        if folder_name == 'real':
            # if pre_folder == 1:
            #     continue
            y.append(1)
            count_real += len(_downsampled)
            pre_folder = 1
        elif folder_name in ALL_DATASET:
            # if pre_folder == 0:
            #     continue
            y.append(0)
            count_fake += len(_downsampled)
            pre_folder = 0
        else:
            raise ValuesError(f"Cannot found in {folder_name} our dataset")

        X[str(idx)] = _downsampled

        
    print('Real Frames:', count_real, 'Fake Frames:', count_fake)
    labels = []
    for i in range(0, count_real):
        labels.append(1)
    for i in range(0, count_fake):
        labels.append(0)
    y_ints = [v.argmax() for v in to_categorical(labels)]

    class_weights = compute_class_weight(class_weight ='balanced', classes=np.unique(y_ints), y=y_ints)
    print('Class Weights:', class_weights)
    return X, y, class_weights

In [4]:

from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from tqdm import tqdm
import numpy as np
import pandas as pd

def find_best_threshold(scores, labels):
        fpr, tpr, thresholds = roc_curve(labels, scores)
        roc_auc = auc(fpr, tpr)

        best_acc = 0
        best_thresh = None
        for i, thresh in enumerate(thresholds):
            # compute accuracy for this threshold
            pred_labels = [1 if s >= thresh else 0 for s in scores]
            acc = sum([1 if pred_labels[j] == labels[j] else 0 for j in range(len(labels))]) / len(labels)

            if acc > best_acc:
                best_acc = acc
                best_thresh = thresh
        return best_thresh, roc_auc


def eval_state(probs, labels, thr):
    predict = probs >= thr
    labels = np.array(labels)
    TN = np.sum((labels == 0) & (predict == False))
    FN = np.sum((labels == 1) & (predict == False))
    FP = np.sum((labels == 0) & (predict == True))
    TP = np.sum((labels == 1) & (predict == True))
    return TN, FN, FP, TP

def evaluate(pretrained_model, data_iter):
    # predictions = pretrained_model.predict_generator(data_iter, verbose=1)
    # prob_list = predictions[:, 1]
    # label_list = data_iter.classes
    
    shape_pred_count = 0
    shape_input_count = 0
    prob_list = []
    label_list = []
    for batch_x, batch_y in tqdm(data_iter):
        # Process the batch inputs and labels
        # ...
        predictions = pretrained_model.predict(batch_x)
        shape_pred_count += len(predictions)
        shape_input_count += len(batch_y)
        
        prob_list.append(predictions)
        label_list.append(batch_y)
        
    prob_list = np.concatenate(prob_list)[:, 1]
    label_list = np.concatenate(label_list)[:, 1].astype(np.int32)

    print("At threshold = 0.5")
    best_thresh = 0.5
    TN, FN, FP, TP = eval_state(probs=prob_list, labels=label_list, thr=best_thresh)
    if (FN + TP == 0):
        FRR = 1.0
        FAR = FP / float(FP + TN)
        TPR = 0
    elif(FP + TN == 0):
        FAR = 1.0
        FRR = FN / float(FN + TP)
        TPR = TP / float(TP + FN)
    else:
        FAR = FP / float(FP + TN)
        FRR = FN / float(FN + TP)
        TPR = TP / float(TP + FN)

    ACC = (TN + TP) / (TN + FN + FP + TP)
    HTER = (FAR + FRR) / 2.0
    print(f"HTER: {HTER*100:.2f}")
    print(f"FAR: {FAR*100:.2f}")
    print(f"TPR: {TPR*100:.2f}")



    np.save('pred_tmp.npy', prob_list)
    np.save('lb_tmp.npy', label_list)
    print("Max values", len(prob_list), np.max(prob_list), np.min(prob_list), np.max(label_list), np.min(label_list))


    best_thresh, AUC = find_best_threshold(scores=prob_list, labels=label_list)
    print(f"At best threshold = {best_thresh:.4f}")
    TN, FN, FP, TP = eval_state(probs=prob_list, labels=label_list, thr=best_thresh)
    if (FN + TP == 0):
        FRR = 1.0
        FAR = FP / float(FP + TN)
        TPR = 0
    elif(FP + TN == 0):
        FAR = 1.0
        FRR = FN / float(FN + TP)
        TPR = TP / float(TP + FN)
    else:
        FAR = FP / float(FP + TN)
        FRR = FN / float(FN + TP)
        TPR = TP / float(TP + FN)


    HTER = (FAR + FRR) / 2.0
    ACC_best = (TN + TP) / (TN + FN + FP + TP)
    print(f"HTER: {HTER*100:.2f}")
    print(f"FAR: {FAR*100:.2f}")
    print(f"TPR: {TPR*100:.2f}")
    print(f"AUC: {AUC*100:.2f}")
    return (ACC*100, ACC_best*100, AUC*100)

In [5]:
# define the model
model = load_model('/media/data1/binh/DeepfakeCampaign/pretrained_checkpoints/clrnet/CLR_ALL_Ebest.hdf5')
is_training = False
top_k_layers=120
model,df=FreezeBatchNormalization(is_training,top_k_layers,model)
print_summary(model, line_length=150, positions=None, print_fn=None)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                

### generated dataset test

In [34]:
frames_per_video_per_batch=5
frames_per_video=10
image_size=128
test_video_per_batch=10
data_augmentation=False
training=False


print("TEST UPON SETTING")
out_results = pd.DataFrame({
    "Dataset":[], "Acc": [], "Acc_best": [], "AUC": []
})
for data_name in ("DeepFaceLab", "Dfaker", "Faceswap", "FOM_Animation", "FOM_Faceswap", "FSGAN", "LightWeight"):

    X_val,y_val,class_weights_val=create_sequence([f'/media/data2/binh/CSIRO/cropped_faces/generated_data/image/{data_name}/',
                                               '/media/data2/binh/CSIRO/cropped_faces/generated_data/image/real/'],frames_per_video_per_batch,frames_per_video)

    val_it=DFVDSequence(X_val,y_val,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation,training)
 
    ACC, ACC_best, AUC = evaluate(pretrained_model=model, data_iter=val_it)
    out_results = out_results.append({"Dataset":data_name, 
                                        "Acc": np.round(ACC,2), 
                                        "Acc_best": np.round(ACC_best,2), 
                                        "AUC": np.round(AUC,2)}, ignore_index=True)
    
out_results = out_results.append({"Dataset":"Avg", 
                                    "Acc": f"{out_results.Acc.mean():.2f} ({out_results.Acc.std():.2f})", 
                                    "Acc_best": f"{out_results.Acc_best.mean():.2f} ({out_results.Acc_best.std():.2f})", 
                                    "AUC": f"{out_results.AUC.mean():.2f} ({out_results.AUC.std():.2f})"}, 
                                    ignore_index=True)
out_results.to_csv(f"predictions/clrnet.csv", index=False)




TEST UPON SETTING
Total Video Folders Found (Real + Fake): 81


81it [00:00, 254.11it/s]


Real Frames: 530 Fake Frames: 280
Class Weights: [1.44642857 0.76415094]


 94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏         | 16/17 [00:36<00:02,  2.20s/it]

*,

...

Real Frames: 530 Fake Frames: 280
Class Weights: [1.44642857 0.76415094]


 94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏         | 16/17 [00:36<00:02,  2.57s/it]

*,

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:39<00:00,  2.33s/it]

At threshold = 0.5
HTER: 23.79
FAR: 26.67
TPR: 79.09
Max values 170 0.9986524 0.00051796273 1 0
At best threshold = 0.1650
HTER: 24.24
FAR: 36.67
TPR: 88.18
AUC: 83.67


## in the wild dataset test

In [19]:
frames_per_video_per_batch=5
frames_per_video=10
image_size=128
test_video_per_batch=10
data_augmentation=False
training=False


print("TEST UPON SETTING")
out_results = pd.DataFrame({
    "Dataset":[], "Acc": [], "Acc_best": [], "AUC": []
})


X_val, y_val, class_weights_val = create_sequence([f'/media/data2/binh/CSIRO/collected_data_sok_single/fake/',
                                            '/media/data2/binh/CSIRO/collected_data_sok_single/real/'],
                                            frames_per_video_per_batch,
                                            frames_per_video)

val_it=DFVDSequence(X_val,y_val,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation,training)

ACC, ACC_best, AUC = evaluate(pretrained_model=model, data_iter=val_it)
out_results = out_results.append({"Dataset":"inthewild", 
                                    "Acc": np.round(ACC,2), 
                                    "Acc_best": np.round(ACC_best,2), 
                                    "AUC": np.round(AUC,2)}, ignore_index=True)

out_results.to_csv(f"predictions/clrnet(in the wild).csv", index=False)


TEST UPON SETTING
Including  /media/data2/binh/CSIRO/collected_data_sok_single/fake/
Including  /media/data2/binh/CSIRO/collected_data_sok_single/real/
Total Video Folders Found (Real + Fake): 1896


1896it [00:09, 196.15it/s]


Real Frames: 5130 Fake Frames: 13830
Class Weights: [0.68546638 1.84795322]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 379/380 [16:54<00:02,  2.73s/it]

*,

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [16:56<00:00,  2.67s/it]


At threshold = 0.5
HTER: 46.07
FAR: 36.48
TPR: 44.33
Max values 3800 0.999493 0.00014698974 1 0
At best threshold = 1.9995
HTER: 50.00
FAR: 0.00
TPR: 0.00
AUC: 55.25
